# Haciendo la sintonización fina de BERT para el dataset

El notebook esta diseñado para correr en colab, por lo que se incluyeron los comandos de instalación para librerías no presentes por defecto

In [1]:
!pip install transformers[torch]
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00


In [17]:
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import pandas as pd
import datasets
import transformers
from transformers import (AutoTokenizer, DataCollatorWithPadding, AutoConfig, AutoModel,
                          AutoModelForSequenceClassification, BertConfig, Trainer, BertForSequenceClassification)
from transformers import TrainingArguments
import torch.nn as nn
import torch
import numpy as np

Los tokens especiales son:

|PAD| = 21

Los normales son:

{"other":0, "1200":1, "1300":2, "1400":3, "1600":4, "1700":5, "1800":6, "1900":7, "2100":8, "2400":9,
            "2405":10, "2600":11, "2800":12, "2805":13, "AR-10":14, "AR-20":15, "AR-30":16, "AR-40":17,
            "AR-50":18, "AR-70":19, "AR-90":20}

En el dataset tenemos este número de cada clase:

6: 377

12: 252

5: 167

0: 117

11: 69

3: 30

4: 29

9: 20

8: 18

1: 15

15: 14

16: 9

14: 6

18: 4

7: 4

13: 3

19: 2

20: 1

17: 1

10: 1

In [4]:
checkpoint = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
config = BertConfig(vocab_size=22, max_position_embeddings=1024) #Limitamos el vocab_size = 22 (tipos de productos)
                                                                 #y una secuencia máxima de 1024 productos
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 1024,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.32.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 22
}

In [6]:
model = AutoModelForSequenceClassification.from_config(config)
model.classifier = nn.Linear(768, 21, bias=True) #Se modifica la última capa para que tenga 21 clases
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(22, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [7]:
#Prueba para ver como es que el modelo saca las predicciones
input = torch.tensor([[0, 0, 1, 2, 2, 3, 21]]).to(device)
softmax = torch.nn.Softmax(dim=1)
softmax(model(input)[0]), model(input)[0][0]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


(tensor([[0.0390, 0.0507, 0.0326, 0.0648, 0.0571, 0.0626, 0.0363, 0.0551, 0.0452,
          0.0410, 0.0482, 0.0522, 0.0558, 0.0487, 0.0407, 0.0442, 0.0272, 0.0436,
          0.0517, 0.0388, 0.0646]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([-0.3686, -0.3910, -0.6431, -0.1695,  0.0962,  0.0329, -0.1564, -0.1221,
          0.1505,  0.0505,  0.1457,  0.2522,  0.2082,  0.1718, -0.0032, -0.1714,
         -0.4201,  0.0382,  0.0852, -0.0935,  0.2232], device='cuda:0',
        grad_fn=<SelectBackward0>))

In [8]:
#Cargamos el dataset
df = pd.read_hdf('data.h5', key='df')
df['X_tuple'] = df.X.apply(tuple)
df['y_tuple'] = df.y.apply(tuple)
df = df.drop_duplicates(subset=['X_tuple', 'y_tuple']) #Eliminamos los datos replicados X->y
df = df.drop(labels=['FE_EMIT', 'FE_ENTREGA', 'PEDIDO', 'X_tuple', 'y_tuple'], axis=1) #Eliminamos columnas que no son útiles
df['y'] = df['y'].apply(lambda x: x[0])
df

,X,y
3503,[0],0
3502,[6],6
3499,"[12, 0]",12
3500,"[12, 0]",0
3497,"[6, 12]",6
...,...,...
31,"[6, 6, 6, 6, 12, 12, 12, 0, 12, 12, 12, 12, 12...",0
22,"[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...",12
17,"[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...",11
10,"[12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 1...",12


In [9]:
#Definimos funciones que son necesarias para el input de BERT, consultese: https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt para más información
from typing import Union
#Función para padear las secuencias
def pad(max_len:int, sequence:Union[list, np.array], pad_token:int=21):
  padding = np.full(max_len-len(sequence), pad_token)
  padded_seq = np.concatenate((sequence, padding))
  return padded_seq.tolist()

#Función para crear las máscaras de atención
def attention_mask(pad_sequence:Union[list, np.array], pad_token:int=21):
  seq = np.asarray(pad_sequence, dtype=int)
  attention_mask = np.asarray(seq != pad_token, dtype=int)
  return attention_mask.tolist()

#Función para crear los arreglos que especifican el tipo de token
def token_types(pad_sequence:Union[list, np.array]):
  return np.zeros(len(pad_sequence), dtype=int).tolist()

In [10]:
#Preparamos el df para BERT con las columnas que requiere('idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids')
max_len = df.X.apply(lambda x: len(x))
max_len = max(max_len)
dt = df.copy(True)

dt.insert(0, 'idx', np.full(len(df), np.nan))
dt.insert(1, 'label', dt.y)
dt.insert(2, 'input_ids', np.full(len(df), np.nan))
dt.insert(3, 'attention_mask', np.full(len(df), np.nan))
dt.insert(4, 'token_type_ids', np.full(len(df), np.nan))
data = dt.to_numpy()

for i in range(len(data)):
  data[i][2] = pad(max_len, data[i][5], pad_token=21)
  data[i][3] = attention_mask(data[i][2], pad_token=21)
  data[i][4] = token_types(data[i][2])
dt['input_ids'] = data[:, 2]
dt['attention_mask'] = data[:, 3]
dt['token_type_ids'] = data[:, 4]
df = dt.copy(True)
df

,idx,label,input_ids,attention_mask,token_type_ids,X,y
3503,NaN,0,"[0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0],0
3502,NaN,6,"[6, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[6],6
3499,NaN,12,"[12, 0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 0]",12
3500,NaN,0,"[12, 0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 0]",0
3497,NaN,6,"[6, 12, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 12]",6
...,...,...,...,...,...,...,...
31,NaN,0,"[6, 6, 6, 6, 12, 12, 12, 0, 12, 12, 12, 12, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 6, 6, 6, 12, 12, 12, 0, 12, 12, 12, 12, 12...",0
22,NaN,12,"[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...",12
17,NaN,11,"[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 12, 12, 12, 12, 11, 11, 11, 11, 12, 12, 1...",11
10,NaN,12,"[12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 1...",12


In [11]:
dt = df.sample(frac = 1).reset_index(drop=True) #Mezclamos el df
# El split es 80/20, sin dataset de testeo
train = int(len(dt)*0.8)
tdf = dt[:train]
vdf = dt[train:]
#Agregamos finalmente el idx a cada elemento
tdf['idx'] = tdf.reset_index(drop=True).index.values
vdf['idx'] = vdf.reset_index(drop=True).index.values

/tmp/ipykernel_466/2206189650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['idx'] = tdf.reset_index(drop=True).index.values
/tmp/ipykernel_466/2206189650.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vdf['idx'] = vdf.reset_index(drop=True).index.values


In [12]:
#Se pasa todo a un datasetdict porque así trabajan los modelos de HuggingFace
tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

ds

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids', 'X', 'y'],
        num_rows: 911
    })
    validation: Dataset({
        features: ['idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids', 'X', 'y'],
        num_rows: 228
    })
})

In [14]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
#Creamos un CustomTrainer que deriva del Trainer, ya que el costo del Trainer base lanza errores
class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                token_type_ids=inputs['token_type_ids']
            )
            labels = torch.zeros(outputs['logits'].shape)
            labels[0, inputs['labels']] = 1
            loss = nn.BCEWithLogitsLoss()(outputs['logits'].to(device),
                                             labels.to(device))
            return (loss, outputs) if return_outputs else loss

In [18]:
#Especificamos los argumentos de entrenamiento
training_args = TrainingArguments("test-trainer",
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  learning_rate = 6e-06,
                                  evaluation_strategy="epoch",
                                  lr_scheduler_type = transformers.get_linear_schedule_with_warmup(optimizer = transformers.AdamW(),
                                                                                                   num_warmup_steps = 100,
                                                                                                   num_training_steps = 13670,
                                                                                                   last_epoch = -1)
                                  )

TypeError: AdamW.__init__() missing 1 required positional argument: 'params'

In [18]:
#Inicializamos el CustomTrainer
trainer = CustomTrainer(
    model,
    training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    compute_metrics=compute_metrics
)

In [19]:
#Entrenamos el modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.212800,0.161694,0.157895
2,0.240200,0.155238,0.333333
3,0.229500,0.127802,0.219298
4,0.215300,0.136166,0.157895
5,0.212900,0.128059,0.333333
6,0.207700,0.151103,0.157895
7,0.202800,0.114683,0.219298
8,0.194700,0.128272,0.219298
9,0.180600,0.174288,0.333333
10,0.173400,0.152066,0.333333


TrainOutput(global_step=13665, training_loss=0.19056692461892444, metrics={'train_runtime': 1880.811, 'train_samples_per_second': 7.265, 'train_steps_per_second': 7.265, 'total_flos': 2331798060683880.0, 'train_loss': 0.19056692461892444, 'epoch': 15.0})

In [1]:
model

NameError: name 'model' is not defined

In [30]:
AutoModel.from_pretrained("test-trainer/checkpoint-13500/")

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(22, 768, padding_idx=0)
    (position_embeddings): Embedding(1024, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

### Medir la precisión del modelo

In [ ]:
y_train = trainer.predict(ds["train"])
print(y_train.predictions.shape, y_train.label_ids.shape)

In [ ]:
y_test = trainer.predict(ds["validation"])
print(y_test.predictions.shape, y_test.label_ids.shape)

In [ ]:
import numpy as np

train_pred = np.argmax(y_train.predictions, axis=-1)
test_pred = np.argmax(y_test.predictions, axis=-1)

In [ ]:
import evaluate

metric = evaluate.load('accuracy')
metric.compute(predictions=train_pred, references=y_train.label_ids), metric.compute(predictions=test_pred, references=y_test.label_ids)